In [1]:
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import os
import pymongo
from bson.objectid import ObjectId
from bson.dbref import DBRef
from Bio import Entrez
from Bio import GenBank

In [2]:
#Connecting to mongodb and getting db
client = pymongo.MongoClient("localhost", 27017, username="admin", password="admin")
db = client['taller_final']

# Transposable elements

In [3]:
#Reading TEs file and preparing TE collection
te_anno_headers = ['seqid','source','sequence_ontology', 'start', 
                   'end', 'score', 'strand', 'phase', 'attributes']
te_anno = pd.read_csv('./ptg000002l.fasta.TEanno.gff3.gz', sep='\t', skiprows=6, header=None)
te_anno.columns = te_anno_headers
te_ids = []
for i in range(len(te_anno)):
    attributes_info = te_anno.iloc[i]["attributes"].split(';')
    te_anno.at[i,'classification'] = attributes_info[2].split('=')[1]
    te_anno.at[i,'sequence_ontology_v2'] = attributes_info[3].split('=')[1]
    te_anno.at[i,'identity'] = attributes_info[4].split('=')[1]
    te_anno.at[i,'method'] = attributes_info[5].split('=')[1]
    te_ids.append(te_anno.iloc[i]["seqid"]+'_'+str(i))
del te_anno['attributes']
te_anno.insert(0, 'te_id', te_ids)
te_anno.head()

,te_id,seqid,source,sequence_ontology,start,end,score,strand,phase,classification,sequence_ontology_v2,identity,method
0,ptg000002l_0,ptg000002l,EDTA,Gypsy_LTR_retrotransposon,15839,17218,4477,+,.,LTR/Gypsy,SO:0002265,0.753,homology
1,ptg000002l_1,ptg000002l,EDTA,Gypsy_LTR_retrotransposon,17520,19985,12445,+,.,LTR/Gypsy,SO:0002265,0.835,homology
2,ptg000002l_2,ptg000002l,EDTA,Gypsy_LTR_retrotransposon,19987,20451,2697,+,.,LTR/Gypsy,SO:0002265,0.836,homology
3,ptg000002l_3,ptg000002l,EDTA,Gypsy_LTR_retrotransposon,20481,24091,9089,+,.,LTR/Gypsy,SO:0002265,0.883,homology
4,ptg000002l_4,ptg000002l,EDTA,Gypsy_LTR_retrotransposon,24092,24877,5031,+,.,LTR/Gypsy,SO:0002265,0.872,homology


In [4]:
#Reading chromosome
chromosome = ''
for record in SeqIO.parse("./ptg000002l.fasta", "fasta"):
    chromosome = record

In [5]:
#Adding TEs sequences to df
for i in range(len(te_anno)):
    start = te_anno.iloc[i]['start']
    end = te_anno.iloc[i]['end']+1
    te_anno.at[i, 'sequence'] = str(chromosome.seq[start:end])

### Saving TEs records in mongo

In [6]:
# ##Saving TEs records
collection = db['tes']
tes_dict = te_anno.to_dict("records")
collection.insert_many(tes_dict)

# Genes and proteins

In [15]:
#Reading blast with predicted proteins
blast_prot = pd.read_csv('./Blast_NR.abinitio_prot_Dm.tab', sep='\t', header=None)
blast_prot_headers = ['query_accession', 'target_accession', 'sequence_identity', 'length', 'mismatches', 
                      'gap_openings', 'query_start', 'query_end', 'target_start', 'target_end', 
                     'e_value', 'bit_score']
blast_prot.columns = blast_prot_headers
blast_predicted_proteins_ids = []
for i in range(len(blast_prot)):
    query = blast_prot.iloc[i]["target_accession"]
    !esearch -db protein -query {query} | efetch -format gb > result.gb
    with open("result.gb") as handle:
        try:
            record = GenBank.read(handle)
            blast_prot["gb_info"] = str(record)
        except:
            blast_prot["gb_info"] = ""
    blast_predicted_proteins_ids.append(blast_prot.iloc[i]["query_accession"]+'_'+str(i))
blast_prot.insert(0, 'blast_prot_pred_id', blast_predicted_proteins_ids)
blast_prot.head()

curl: (35) OpenSSL SSL_connect: Connection reset by peer in connection to eutils.ncbi.nlm.nih.gov:443 
>78 ERROR: >78 curl command failed ( Wed Oct 19 02:33:04 PM -05 2022 ) with: 35>78
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi -d retmax=0&usehistory=y&db=protein&term=XP_002512792.2&tool=edirect&edirect=16.2&edirect_os=Linux&email=breaze%40breaze>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -usehistory y -db protein -term XP_002512792.2 -tool edirect -edirect 16.2 -edirect_os Linux -email breaze@breaze
EMPTY RESULT
SECOND ATTEMPT
curl: (28) SSL connection timeout
>78 ERROR: >78 curl command failed ( Wed Oct 19 02:55:55 PM -05 2022 ) with: 28>78
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi -d retmax=0&usehistory=y&db=protein&term=PKA65666.1&tool=edirect&edirect=16.2&edirect_os=Linux&email=breaze%40breaze>78
nquire -url https://eutils.ncbi.nlm.nih.gov/entrez/eutils/ esearch.fcgi -retmax 0 -useh

,blast_prot_pred_id,query_accession,target_accession,sequence_identity,length,mismatches,gap_openings,query_start,query_end,target_start,target_end,e_value,bit_score,gb_info
0,g4.t1_0,g4.t1,XP_031739819.1,84.9,225,33,1,1,224,1,225,8.400000e-104,388.3,LOCUS XP_030957800 1121 aa PROTEIN ...
1,g4.t1_1,g4.t1,KAH7528545.1,81.6,147,27,0,1,147,9,155,2.700000e-62,250.4,LOCUS XP_030957800 1121 aa PROTEIN ...
2,g6.t1_2,g6.t1,PNS97074.1,80.2,329,63,2,109,437,1,327,7.400000e-157,564.7,LOCUS XP_030957800 1121 aa PROTEIN ...
3,g7.t1_3,g7.t1,MBA0694754.1,80.4,465,68,2,1105,1567,1,444,1.200000e-220,778.9,LOCUS XP_030957800 1121 aa PROTEIN ...
4,g8.t1_4,g8.t1,KAH7572690.1,80.0,105,21,0,1,105,1,105,3.200000e-40,175.3,LOCUS XP_030957800 1121 aa PROTEIN ...


In [16]:
# ##Saving records of blast with predicted proteins 
collection = db['blast_predicted_proteins']
blast_pred_prot_dict = blast_prot.to_dict("records")
collection.insert_many(blast_pred_prot_dict)

In [17]:
#Reading pfam file with protein domains
domains = pd.read_csv('./augustus.abinitio.pfam_results.txt', delim_whitespace=True, skiprows=28, header=None)
domains_headers = ['seqid','alignment_start','alignment_end', 'envelope_start', 
                   'envelope_end', 'hmm_acc', 'hmm_name', 'type', 'hmm_start', 'hmm_end', 'hmm_length',
                  'bit_score', 'e_value', 'significance', 'clan']
domains.columns = domains_headers
prot_domain_ids = []
for i in range(len(domains)):
    prot_domain_ids.append(domains.iloc[i]["seqid"]+'_'+str(i))
domains.insert(0, 'prot_domain_id', prot_domain_ids)
domains.head()

,prot_domain_id,seqid,alignment_start,alignment_end,envelope_start,envelope_end,hmm_acc,hmm_name,type,hmm_start,hmm_end,hmm_length,bit_score,e_value,significance,clan
0,g4.t1_0,g4.t1,173,313,154,313,PF13359.9,DDE_Tnp_4,Domain,24,158,158,51.9,7.200000e-14,1,CL0219
1,g5.t1_1,g5.t1,52,119,52,144,PF00300.25,His_Phos_1,Domain,1,68,194,29.8,4.700000e-07,1,CL0071
2,g6.t1_2,g6.t1,64,260,61,260,PF05634.14,APO_RNA-bind,Family,4,200,200,347.5,1.800000e-104,1,No_clan
3,g6.t1_3,g6.t1,300,415,291,427,PF05634.14,APO_RNA-bind,Family,66,181,200,82.2,3.300000e-23,1,No_clan
4,g7.t1_4,g7.t1,144,190,144,192,PF13041.9,PPR_2,Repeat,1,47,50,31.6,1.500000e-07,1,CL0020


In [18]:
##Saving protein domains
collection = db['protein_domains']
domains_dict = domains.to_dict("records")
collection.insert_many(domains_dict)

In [19]:
def searchBlastPredictedProteins(query):
    collection = db['blast_predicted_proteins']
    blast_pred_prot_query = {"query_accession": query}
    mydoc = collection.find(blast_pred_prot_query)
    blast_pred_prot_ref = []
    for bl in mydoc:
        blast_pred_prot_ref.append(bl["_id"])
    return blast_pred_prot_ref

In [20]:
def searchProteinDomain(query):
    collection = db['protein_domains']
    protein_domain_query = {"seqid": query}
    mydoc = collection.find(protein_domain_query)
    protein_domain_ref = []
    for prt in mydoc:
        protein_domain_ref.append(prt["_id"])
    return protein_domain_ref

In [21]:
#Reading and processing genes collection
genes = pd.DataFrame()
genes_ids = []
genes_seq = []
for gen in SeqIO.parse("./augustus.abinitio.aa", "fasta"):
    genes_ids.append(gen.id)
    genes_seq.append(str(gen.seq))
genes["gen_id"] = genes_ids
genes["seq"] = genes_seq
temp_blast_refs = []
temp_domain_refs = []
for i in range(len(genes)):
    temp_blast_refs.append(searchBlastPredictedProteins(genes.iloc[i]["gen_id"]))
    temp_domain_refs.append(searchProteinDomain(genes.iloc[i]["gen_id"]))
genes["blast_predicted_prot"] = temp_blast_refs
genes["protein_domains"] = temp_domain_refs
genes.head()

,gen_id,seq,blast_predicted_prot,protein_domains
0,g1.t1,MDQATELEREDNALLKQFLDLKPIKYKGIGNASLAEDWLAEMNKIF...,[],[]
1,g2.t1,MKVHVPDSVGLTTVLHSVWKSEVHIPDSAGPTTALHSEWHDRIRLG...,[],[]
2,g3.t1,MRLHVPDSAGPTTALQSEKVHVPDSAGLTTALHSEWHVPDSAGLTT...,[],[]
3,g4.t1,MESSDDEKDLLCGKSLVKDLAYNLVPTGIKFVDEVLNGPNERCLEN...,"[635059954621c31f518fbb39, 635059954621c31f518...",[63505a084621c31f518fc8f2]
4,g5.t1,MHPAACKVQPLPLFCPKSIRKSKSTKSLRLVTETPTAEDQASPSPI...,[],[63505a084621c31f518fc8f3]


In [22]:
##Saving records predicted genes
collection = db['predicted_genes']
predicted_genes_dict = genes.to_dict("records")
collection.insert_many(predicted_genes_dict)